In [1]:


import pandas as pd
import dask.dataframe as dd
import dask.array as da
import numpy as np
from dask.distributed import Client, LocalCluster
import replicaEVSE.sql_wrapper_functions as sql
import replicaEVSE.load_curve as sim
import replicaEVSE.geography as geo
import os
import requests
import geopandas as gpd
import datetime
from dask.diagnostics import ProgressBar

%reload_ext autoreload
%autoreload 2

# pbar = ProgressBar()
# pbar.register()

pd.set_option('display.max_columns', None)

datadir = '../../data'

with open('../src/metadata/MySQLpwd.txt', 'r') as f:
    pw = f.readlines()
pw = pw[0]
# connection_replica = sql.create_db_connection("localhost", "root", pw, 'replica')

pd.set_option('display.max_columns', None)



In [2]:
# read in the joined data

merged_ddf = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips.parquet'))

## take a small sample to build with

make it a dask DataFrame so the experience is the same

In [55]:
#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
# ddf = merged_ddf.sample(frac=0.0000001)
ddf = merged_ddf.head(10000)
ddf = dd.from_pandas(ddf, chunksize=1000).reset_index()
print(len(ddf))

# right now, only look at private auto trips
ddf = ddf.loc[ddf['mode'] == 'PRIVATE_AUTO']
print(len(ddf))




print(ddf.index.head())
# sort on person_id and start_time
ddf = ddf.sort_values(by=['person_id', 'start_time', 'weekday']).reset_index(drop=True)



ddf.to_parquet(datadir+'/wa_pop_and_trips_subsample.parquet', engine='pyarrow', compression='snappy')

10000
5227
Int64Index([1, 2, 6, 8, 10], dtype='int64')


### need to optimize `create_charging_events()` 

I will use the first few users who have the most trips to make a toy example. We can them apply this to a more realistic sample. 

which users took the most trips:

In [56]:
cond = ddf['person_id'].isin(['2712046531787096716', '10382476570347818940', '3739083454226448410'])
cond = cond & (ddf['weekday'] == 'thursday')
ddf.loc[cond]['person_id'].value_counts().compute().head(3)

3739083454226448410     4
10382476570347818940    2
2712046531787096716     1
Name: person_id, dtype: int64

In [57]:
# Sort the dataframe by person and start time
trips = ddf.loc[cond].sort_values(by=['person_id', 'start_time'])
df = trips[['person_id', 'start_time', 'end_time']].compute()

sim.calculate_stop_duration(df)

,person_id,start_time,end_time,stop_duration
0,10382476570347818940,0 days 07:52:44,0 days 07:58:41,0 days 11:07:40
1,10382476570347818940,0 days 19:06:21,0 days 19:16:55,0 days 12:35:49
2,2712046531787096716,0 days 12:15:01,0 days 12:41:24,0 days 23:33:37
3,3739083454226448410,0 days 09:06:41,0 days 09:12:16,0 days 03:13:53
4,3739083454226448410,0 days 12:26:09,0 days 12:30:57,0 days 04:01:04
5,3739083454226448410,0 days 16:32:01,0 days 16:38:50,0 days 00:20:57
6,3739083454226448410,0 days 16:59:47,0 days 17:11:20,0 days 15:55:21


In [58]:
ddf_with_stop_duration = ddf.map_partitions(sim.calculate_stop_duration)

In [59]:
ddf_with_stop_duration.compute()

,index,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type,stop_duration
0,4199,7952205927434267893,10000366922410938523,PRIVATE_AUTO,HOME,EAT,0 days 12:41:15,0 days 12:43:31,0.602554,nan,530330068002,47.636260,-122.358964,530330067004,47.635745,-122.347078,commercial,retail,commercial,retail,residential,multi_family,residential,multi_family,47.638249,-122.356634,47.636388,-122.346274,thursday,1732374551127832797,530330067004,nan,nan,53033006700,nan,nan,35_64,62.0,M,white,not_hispanic_or_latino,20000_40000,22176.0,notinlf,some_college,not_attending_school,naics72,head_of_household,0,nan,not_working,renter,same_house,5_person,40000_75000,59308.0,nonfamily_single,1,multiple_units,core,english,47.636388,-122.346274,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME,0 days 23:57:44
1,3575,506829784905170377,10001830118790204821,PRIVATE_AUTO,HOME,EAT,0 days 19:43:00,0 days 19:56:05,1.011157,nan,530610516012,47.844719,-122.299453,530610516021,47.834790,-122.308797,commercial,retail,commercial,retail,mixed_use,single_family,residential,single_family,47.840838,-122.300091,47.834503,-122.313617,thursday,3635700334446387909,530610516021,530350805002,530610516014,53061051602,53035080500,53061051601,18_24,18.0,M,white,not_hispanic_or_latino,0_20000,216.0,employed,high_school,school,naics722513,child,0,nan,carpool,renter,same_house,3_person,40000_75000,41273.0,family_single,2,multiple_units,core,english,47.834503,-122.313617,NaN,NaN,47.845652,-122.315406,worked_in_person,HOME,0 days 23:46:55
2,4816,13722390883687412311,10002290586277908152,PRIVATE_AUTO,HOME,EAT,0 days 18:40:12,0 days 18:46:32,2.150550,nan,530330323092,47.675673,-122.126007,530330228023,47.663252,-122.138520,mixed_use,office,commercial,retail,residential,single_family,residential,single_family,47.670172,-122.111026,47.664865,-122.135813,thursday,4129290245843427363,530330228023,nan,nan,53033022802,nan,nan,25_34,34.0,F,asian,not_hispanic_or_latino,lte_0,0.0,notinlf,advanced_degree,not_attending_school,not_working,spouse,0,nan,not_working,owner,same_house,5_person,125000_plus,319710.0,married_couple,3_plus,single_family,core,asian_pacific,47.664865,-122.135813,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME,0 days 23:53:40
3,247,11325025596515485190,10002759413662457525,PRIVATE_AUTO,WORK,HOME,0 days 07:24:41,0 days 07:37:33,6.867275,nan,530670109103,47.016462,-122.909304,530670112003,47.047701,-122.808723,residential,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,47.016530,-122.908200,47.040000,-122.800000,thursday,3706854826325843051,530670109103,530670112003,nan,53067010910,53067011200,nan,25_34,33.0,F,white,not_hispanic_or_latino,40000_80000,51862.0,employed,high_school,not_attending_school,naics92,spouse,0,nan,driving,owner,same_house,4_person,75000_125000,111288.0,married_couple,3_plus,single_family,core,other,47.016530,-122.908200,47.040000,-122.80000,NaN,NaN,worked_in_person,WORK,0 days 02:14:40
4,4243,2725037000145676387,10002759413662457525,PRIVATE_AUTO,SHOP,WORK,0 days 09:52:13,0 days 09:58:59,1.866797,nan,530670112003,47.047701,-122.808723,5306701120

add total miles per person

In [60]:
# add total miles per person
total_distance_df = ddf.groupby('person_id')['distance_miles'].sum().to_frame('total_distance')
ddf_tot_dist = ddf_with_stop_duration.merge(total_distance_df, on='person_id', how='left')

# Calculate total energy consumption from total miles driven and energy consumption estimate
consumption_kWh_per_mi = 0.3
total_energy = ddf_tot_dist['total_distance']*consumption_kWh_per_mi

def assign_charger_available(df):
    df['charger_power_kW'] = 0
    df['charger_power_kW'] = df['charger_power_kW'].mask(df['charge_type'] == 'HOME', 7.2)
    df['charger_power_kW'] = df['charger_power_kW'].mask(df['charge_type'] == 'WORK', 7.2)
    df['charger_power_kW'] = df['charger_power_kW'].mask(df['charge_type'] == 'PUBLIC', 150)
    return df

# get charger availability
ddf_charger_available = assign_charger_available(ddf_tot_dist)

# Calculate total charge opportunity using stop duration and charger power
ddf_charger_available['charge_opportunity_remaining_kWh'] = ddf_charger_available['stop_duration'].dt.seconds/60/60*df['charger_power_kW']

# Initialize total energy to consume
remaining_energy = total_energy



KeyError: 'charger_power_kW'

In [51]:
ddf_with_stop_duration['stop_duration'].head()

0   0 days 23:57:44
1   0 days 23:46:55
2   0 days 23:53:40
3   0 days 02:14:40
4   0 days 21:25:42
Name: stop_duration, dtype: timedelta64[ns]